# Extract respiratory signals with RAFT

Recurrent All-Pairs Field Transforms (RAFT) is a deep learning model for optical flow estimation. The optical flow directions and magnitudes can be used to extract respiratory signals from videos. This notebook demonstrates how to use RAFT to extract respiratory signals from videos.

In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

subject = 'Proband16'
setting = '101_natural_lighting'

video_path = dataset.get_video_path(subject, setting)

In [ ]:
import respiration.utils as utils
import respiration.extractor.optical_flow_raft as raft

device = utils.get_torch_device()
model = raft.load_model('raft_large', device)

In [ ]:
import torch
import numpy as np
from tqdm.auto import tqdm

# Number of frames that are processed at once
batch_size = 10

param = utils.get_video_params(video_path)

# Make an empty numpy to store the optical flows in the N, 2, H, W format
optical_flows = np.zeros((param.num_frames, 2, param.height, param.width), dtype=np.float32)

for start in tqdm(range(0, param.num_frames, batch_size)):
    # Calculate the number of frames to process in this batch
    num_frames = min(start + batch_size, param.num_frames) - start

    frames, _ = utils.read_video_rgb(video_path, num_frames, start)
    frames = raft.preprocess(frames, device)

    with torch.no_grad():
        # Split the frames into odd and even frames to calculate optical flow on consecutive frames
        flows = model(frames[::2], frames[1::2])

    # Garbage collect...
    del frames

    # Only keep the last flow iteration
    flows = flows[-1]

    for idx in range(flows.shape[0]):
        # Add the optical flow to the numpy array
        optical_flows[start + idx] = flows[idx].cpu().numpy()

In [ ]:
# Store the optical_flows matrix to a file
np.save('optical_flows.npy', optical_flows)